In [ ]:
# ! pip install pydub
# ! pip install audiosegment
# ! pip install resampy
# ! pip install joblib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp310-cp310-linux_x86_64.whl size=81212 sha256=14fda7c9a83bac8b518f6f670016284274d4b74cd0c805eb912242504d679f88
  Stored in directory: /root/.cache/pip/wheels/2a/2b/84/ac7bacfe8c68a87c1ee3dd3c66818a54c71599abf308e8eb35
Successfully built webrtcvad
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


notes chatgpt:

Untuk memperbaiki prediksi pada audio file baru, Anda dapat mencoba beberapa langkah berikut:



In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import librosa
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def check_audio_characteristics(file_path, expected_sample_rate, expected_duration):
    audio, sample_rate = librosa.load(file_path, sr=None)
    duration = len(audio) / sample_rate

    if sample_rate != expected_sample_rate:
        print(f"Perbedaan tingkat sampel audio! Sample rate yang diharapkan: {expected_sample_rate}, Sample rate file: {sample_rate}")
    
    if duration != expected_duration:
        print(f"Perbedaan durasi audio! Durasi yang diharapkan: {expected_duration} detik, Durasi file: {duration} detik")
    
    # Jika terdapat perbedaan lain yang ingin diperiksa, tambahkan di sini
    
    # Jika tidak terdapat perbedaan, cetak pesan bahwa karakteristik audio cocok
    if sample_rate == expected_sample_rate and duration == expected_duration:
        print("Karakteristik audio cocok!")

In [ ]:
# Mendefinisikan direktori data train
train_directory = '/content/drive/MyDrive/Product_Capstone_Materials/Alphabet Classification (Multi-Class)/'

# Inisialisasi variabel expected_sample_rate dan expected_duration
expected_sample_rate = None
expected_duration = None

# Looping untuk setiap file audio di direktori data train
for file_name in os.listdir(train_directory):
    file_path = os.path.join(train_directory, file_name)
    
    # Memeriksa apakah file audio
    if file_name.endswith('.wav'):
        # Membaca file audio dan mendapatkan sample rate dan durasi
        audio, sample_rate = librosa.load(file_path, sr=None)
        duration = len(audio) / sample_rate
        
        # Memperbarui expected_sample_rate jika belum diinisialisasi atau berbeda dengan sample rate saat ini
        if expected_sample_rate is None or expected_sample_rate != sample_rate:
            expected_sample_rate = sample_rate
        
        # Memperbarui expected_duration jika belum diinisialisasi atau berbeda dengan durasi saat ini
        if expected_duration is None or expected_duration != duration:
            expected_duration = duration

# Menampilkan expected_sample_rate dan expected_duration
print('Expected Sample Rate:', expected_sample_rate)
print('Expected Duration:', expected_duration)

Expected Sample Rate: 44100
Expected Duration: 0.8356235827664399


In [ ]:
from IPython.display import Audio

# Specify the path to the audio file
audio_file = '/content/drive/MyDrive/Colab Notebooks/export.wav'
audio_file2 = '/content/drive/MyDrive/Colab Notebooks/export2.wav'
audio_file3 = '/content/drive/MyDrive/Colab Notebooks/export3.wav'
# Play the audio file
Audio(audio_file)

In [ ]:
Audio(audio_file2)

In [ ]:
Audio(audio_file3)

In [ ]:
import random
import librosa
import os
import resampy

# Fungsi untuk ekstraksi fitur menggunakan MFCC
def extract_mfcc(file_path, expected_sample_rate, expected_duration, n_mfcc=13, max_len=60):
    audio, sample_rate = librosa.load(file_path, sr=None)
    duration = len(audio) / sample_rate

    if sample_rate != expected_sample_rate:
        # Melakukan resampling audio jika tingkat sampel berbeda
        audio = resampy.resample(audio, sample_rate, expected_sample_rate)
        sample_rate = expected_sample_rate

    if duration != expected_duration:
        # Melakukan trimming atau padding audio jika durasi berbeda
        target_length = int(expected_duration * sample_rate)
        if len(audio) < target_length:
            # Padding audio dengan nol jika durasi lebih pendek
            audio = np.pad(audio, (0, target_length - len(audio)), 'constant')
        else:
            # Trimming audio jika durasi lebih panjang
            audio = audio[:target_length]

    mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=n_mfcc)
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    return mfcc

# Fungsi untuk ekstraksi fitur menggunakan Spectrogram
def extract_spectrogram(file_path, expected_sample_rate, expected_duration, n_mels=128, n_fft=2048, hop_length=512, max_len=60):
    audio, sample_rate = librosa.load(file_path, sr=None)
    duration = len(audio) / sample_rate

    if sample_rate != expected_sample_rate:
        # Melakukan resampling audio jika tingkat sampel berbeda
        audio = resampy.resample(audio, sample_rate, expected_sample_rate)
        sample_rate = expected_sample_rate

    if duration != expected_duration:
        # Melakukan trimming atau padding audio jika durasi berbeda
        target_length = int(expected_duration * sample_rate)
        if len(audio) < target_length:
            # Padding audio dengan nol jika durasi lebih pendek
            audio = np.pad(audio, (0, target_length - len(audio)), 'constant')
        else:
            # Trimming audio jika durasi lebih panjang
            audio = audio[:target_length]

    spectrogram = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
    spectrogram = librosa.power_to_db(spectrogram)
    if (max_len > spectrogram.shape[1]):
        pad_width = max_len - spectrogram.shape[1]
        spectrogram = np.pad(spectrogram, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        spectrogram = spectrogram[:, :max_len]
    return spectrogram

# Load data dari file CSV
data = pd.read_csv('/content/drive/MyDrive/Product_Capstone_Materials/fullclass.csv')

# Load label encoder dan fitting
label_encoder = LabelEncoder()
label_encoder.fit(data['label'])

# Memuat model yang telah disimpan
model = load_model('/content/drive/MyDrive/Colab Notebooks/modelhijaiyah_5.h5')

# Memasukkan path audio yang ingin diprediksi
file_path = '/content/drive/MyDrive/Colab Notebooks/export.wav'

# Karakteristik audio yang diharapkan
expected_sample_rate = 44100
expected_duration = 0.8356235827664399

# Memeriksa apakah file audio ada dan sesuai dengan karakteristik yang diharapkan
if not os.path.exists(file_path):
    print("File tidak ditemukan!")
else:
    check_audio_characteristics(file_path, expected_sample_rate, expected_duration)

    # Ekstraksi fitur MFCC dan Spectrogram dari data audio
    mfcc_features = extract_mfcc(file_path, expected_sample_rate, expected_duration)
    spectrogram_features = extract_spectrogram(file_path, expected_sample_rate, expected_duration)

    # Ubah dimensi data untuk model CNN
    mfcc_features = mfcc_features[np.newaxis, ..., np.newaxis]
    spectrogram_features = spectrogram_features[np.newaxis, ..., np.newaxis]

    # Prediksi label menggunakan model
    predictions = model.predict([mfcc_features, spectrogram_features])

    # Decode label menggunakan label_encoder
    predicted_label = label_encoder.inverse_transform([np.argmax(predictions)])[0]

    print('Path:', file_path)
    print('Predicted label:', predicted_label)

Perbedaan durasi audio! Durasi yang diharapkan: 0.8356235827664399 detik, Durasi file: 0.4696825396825397 detik


1/1 [==============================] - 0s 145ms/step
Path: /content/drive/MyDrive/Colab Notebooks/export.wav
Predicted label: alif


In [ ]:
import random
import librosa
import os
import resampy

# Fungsi untuk ekstraksi fitur menggunakan MFCC
def extract_mfcc(file_path, expected_sample_rate, expected_duration, n_mfcc=13, max_len=60):
    audio, sample_rate = librosa.load(file_path, sr=None)
    duration = len(audio) / sample_rate

    if sample_rate != expected_sample_rate:
        # Melakukan resampling audio jika tingkat sampel berbeda
        audio = resampy.resample(audio, sample_rate, expected_sample_rate)
        sample_rate = expected_sample_rate

    if duration != expected_duration:
        # Melakukan trimming atau padding audio jika durasi berbeda
        target_length = int(expected_duration * sample_rate)
        if len(audio) < target_length:
            # Padding audio dengan nol jika durasi lebih pendek
            audio = np.pad(audio, (0, target_length - len(audio)), 'constant')
        else:
            # Trimming audio jika durasi lebih panjang
            audio = audio[:target_length]

    mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=n_mfcc)
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    return mfcc

# Fungsi untuk ekstraksi fitur menggunakan Spectrogram
def extract_spectrogram(file_path, expected_sample_rate, expected_duration, n_mels=128, n_fft=2048, hop_length=512, max_len=60):
    audio, sample_rate = librosa.load(file_path, sr=None)
    duration = len(audio) / sample_rate

    if sample_rate != expected_sample_rate:
        # Melakukan resampling audio jika tingkat sampel berbeda
        audio = resampy.resample(audio, sample_rate, expected_sample_rate)
        sample_rate = expected_sample_rate

    if duration != expected_duration:
        # Melakukan trimming atau padding audio jika durasi berbeda
        target_length = int(expected_duration * sample_rate)
        if len(audio) < target_length:
            # Padding audio dengan nol jika durasi lebih pendek
            audio = np.pad(audio, (0, target_length - len(audio)), 'constant')
        else:
            # Trimming audio jika durasi lebih panjang
            audio = audio[:target_length]

    spectrogram = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
    spectrogram = librosa.power_to_db(spectrogram)
    if (max_len > spectrogram.shape[1]):
        pad_width = max_len - spectrogram.shape[1]
        spectrogram = np.pad(spectrogram, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        spectrogram = spectrogram[:, :max_len]
    return spectrogram

# Load data dari file CSV
data = pd.read_csv('/content/drive/MyDrive/Product_Capstone_Materials/fullclass.csv')

# Load label encoder dan fitting
label_encoder = LabelEncoder()
label_encoder.fit(data['label'])

# Memuat model yang telah disimpan
model = load_model('/content/drive/MyDrive/Colab Notebooks/modelhijaiyah_5.h5')

# Memasukkan path audio yang ingin diprediksi
file_path = '/content/drive/MyDrive/Colab Notebooks/export2.wav'

# Karakteristik audio yang diharapkan
expected_sample_rate = 44100
expected_duration = 0.8356235827664399

# Memeriksa apakah file audio ada dan sesuai dengan karakteristik yang diharapkan
if not os.path.exists(file_path):
    print("File tidak ditemukan!")
else:
    check_audio_characteristics(file_path, expected_sample_rate, expected_duration)

    # Ekstraksi fitur MFCC dan Spectrogram dari data audio
    mfcc_features = extract_mfcc(file_path, expected_sample_rate, expected_duration)
    spectrogram_features = extract_spectrogram(file_path, expected_sample_rate, expected_duration)

    # Ubah dimensi data untuk model CNN
    mfcc_features = mfcc_features[np.newaxis, ..., np.newaxis]
    spectrogram_features = spectrogram_features[np.newaxis, ..., np.newaxis]

    # Prediksi label menggunakan model
    predictions = model.predict([mfcc_features, spectrogram_features])

    # Decode label menggunakan label_encoder
    predicted_label = label_encoder.inverse_transform([np.argmax(predictions)])[0]

    print('Path:', file_path)
    print('Predicted label:', predicted_label)

Perbedaan durasi audio! Durasi yang diharapkan: 0.8356235827664399 detik, Durasi file: 1.7647165532879818 detik
1/1 [==============================] - 0s 165ms/step
Path: /content/drive/MyDrive/Colab Notebooks/export2.wav
Predicted label: ba


In [ ]:
import random
import librosa
import os
import resampy

# Fungsi untuk ekstraksi fitur menggunakan MFCC
def extract_mfcc(file_path, expected_sample_rate, expected_duration, n_mfcc=13, max_len=60):
    audio, sample_rate = librosa.load(file_path, sr=None)
    duration = len(audio) / sample_rate

    if sample_rate != expected_sample_rate:
        # Melakukan resampling audio jika tingkat sampel berbeda
        audio = resampy.resample(audio, sample_rate, expected_sample_rate)
        sample_rate = expected_sample_rate

    if duration != expected_duration:
        # Melakukan trimming atau padding audio jika durasi berbeda
        target_length = int(expected_duration * sample_rate)
        if len(audio) < target_length:
            # Padding audio dengan nol jika durasi lebih pendek
            audio = np.pad(audio, (0, target_length - len(audio)), 'constant')
        else:
            # Trimming audio jika durasi lebih panjang
            audio = audio[:target_length]

    mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=n_mfcc)
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    return mfcc

# Fungsi untuk ekstraksi fitur menggunakan Spectrogram
def extract_spectrogram(file_path, expected_sample_rate, expected_duration, n_mels=128, n_fft=2048, hop_length=512, max_len=60):
    audio, sample_rate = librosa.load(file_path, sr=None)
    duration = len(audio) / sample_rate

    if sample_rate != expected_sample_rate:
        # Melakukan resampling audio jika tingkat sampel berbeda
        audio = resampy.resample(audio, sample_rate, expected_sample_rate)
        sample_rate = expected_sample_rate

    if duration != expected_duration:
        # Melakukan trimming atau padding audio jika durasi berbeda
        target_length = int(expected_duration * sample_rate)
        if len(audio) < target_length:
            # Padding audio dengan nol jika durasi lebih pendek
            audio = np.pad(audio, (0, target_length - len(audio)), 'constant')
        else:
            # Trimming audio jika durasi lebih panjang
            audio = audio[:target_length]

    spectrogram = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
    spectrogram = librosa.power_to_db(spectrogram)
    if (max_len > spectrogram.shape[1]):
        pad_width = max_len - spectrogram.shape[1]
        spectrogram = np.pad(spectrogram, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        spectrogram = spectrogram[:, :max_len]
    return spectrogram

# Load data dari file CSV
data = pd.read_csv('/content/drive/MyDrive/Product_Capstone_Materials/fullclass.csv')

# Load label encoder dan fitting
label_encoder = LabelEncoder()
label_encoder.fit(data['label'])

# Memuat model yang telah disimpan
model = load_model('/content/drive/MyDrive/Colab Notebooks/modelhijaiyah_5.h5')

# Memasukkan path audio yang ingin diprediksi
file_path = '/content/drive/MyDrive/Colab Notebooks/export3.wav'

# Karakteristik audio yang diharapkan
expected_sample_rate = 44100
expected_duration = 0.8356235827664399

# Memeriksa apakah file audio ada dan sesuai dengan karakteristik yang diharapkan
if not os.path.exists(file_path):
    print("File tidak ditemukan!")
else:
    check_audio_characteristics(file_path, expected_sample_rate, expected_duration)

    # Ekstraksi fitur MFCC dan Spectrogram dari data audio
    mfcc_features = extract_mfcc(file_path, expected_sample_rate, expected_duration)
    spectrogram_features = extract_spectrogram(file_path, expected_sample_rate, expected_duration)

    # Ubah dimensi data untuk model CNN
    mfcc_features = mfcc_features[np.newaxis, ..., np.newaxis]
    spectrogram_features = spectrogram_features[np.newaxis, ..., np.newaxis]

    # Prediksi label menggunakan model
    predictions = model.predict([mfcc_features, spectrogram_features])

    # Decode label menggunakan label_encoder
    predicted_label = label_encoder.inverse_transform([np.argmax(predictions)])[0]

    print('Path:', file_path)
    print('Predicted label:', predicted_label)


Perbedaan durasi audio! Durasi yang diharapkan: 0.8356235827664399 detik, Durasi file: 0.5061904761904762 detik
1/1 [==============================] - 0s 280ms/step
Path: /content/drive/MyDrive/Colab Notebooks/export3.wav
Predicted label: ba


In [ ]:
import random
import librosa
import os
import resampy

# Fungsi untuk ekstraksi fitur menggunakan MFCC
def extract_mfcc(file_path, expected_sample_rate, expected_duration, n_mfcc=13, max_len=60):
    audio, sample_rate = librosa.load(file_path, sr=None)
    duration = len(audio) / sample_rate

    if sample_rate != expected_sample_rate:
        # Melakukan resampling audio jika tingkat sampel berbeda
        audio = resampy.resample(audio, sample_rate, expected_sample_rate)
        sample_rate = expected_sample_rate

    if duration != expected_duration:
        # Melakukan trimming atau padding audio jika durasi berbeda
        target_length = int(expected_duration * sample_rate)
        if len(audio) < target_length:
            # Padding audio dengan nol jika durasi lebih pendek
            audio = np.pad(audio, (0, target_length - len(audio)), 'constant')
        else:
            # Trimming audio jika durasi lebih panjang
            audio = audio[:target_length]

    mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=n_mfcc)
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    return mfcc

# Fungsi untuk ekstraksi fitur menggunakan Spectrogram
def extract_spectrogram(file_path, expected_sample_rate, expected_duration, n_mels=128, n_fft=2048, hop_length=512, max_len=60):
    audio, sample_rate = librosa.load(file_path, sr=None)
    duration = len(audio) / sample_rate

    if sample_rate != expected_sample_rate:
        # Melakukan resampling audio jika tingkat sampel berbeda
        audio = resampy.resample(audio, sample_rate, expected_sample_rate)
        sample_rate = expected_sample_rate

    if duration != expected_duration:
        # Melakukan trimming atau padding audio jika durasi berbeda
        target_length = int(expected_duration * sample_rate)
        if len(audio) < target_length:
            # Padding audio dengan nol jika durasi lebih pendek
            audio = np.pad(audio, (0, target_length - len(audio)), 'constant')
        else:
            # Trimming audio jika durasi lebih panjang
            audio = audio[:target_length]

    spectrogram = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
    spectrogram = librosa.power_to_db(spectrogram)
    if (max_len > spectrogram.shape[1]):
        pad_width = max_len - spectrogram.shape[1]
        spectrogram = np.pad(spectrogram, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        spectrogram = spectrogram[:, :max_len]
    return spectrogram

# Load data dari file CSV
data = pd.read_csv('/content/drive/MyDrive/Product_Capstone_Materials/fullclass.csv')

# Load label encoder dan fitting
label_encoder = LabelEncoder()
label_encoder.fit(data['label'])

# Memuat model yang telah disimpan
model = load_model('/content/drive/MyDrive/Colab Notebooks/modelhijaiyah_5.h5')

# Memasukkan path audio yang ingin diprediksi
file_path = '/content/drive/MyDrive/Colab Notebooks/export4.wav'

# Karakteristik audio yang diharapkan
expected_sample_rate = 44100
expected_duration = 0.8356235827664399

# Memeriksa apakah file audio ada dan sesuai dengan karakteristik yang diharapkan
if not os.path.exists(file_path):
    print("File tidak ditemukan!")
else:
    check_audio_characteristics(file_path, expected_sample_rate, expected_duration)

    # Ekstraksi fitur MFCC dan Spectrogram dari data audio
    mfcc_features = extract_mfcc(file_path, expected_sample_rate, expected_duration)
    spectrogram_features = extract_spectrogram(file_path, expected_sample_rate, expected_duration)

    # Ubah dimensi data untuk model CNN
    mfcc_features = mfcc_features[np.newaxis, ..., np.newaxis]
    spectrogram_features = spectrogram_features[np.newaxis, ..., np.newaxis]

    # Prediksi label menggunakan model
    predictions = model.predict([mfcc_features, spectrogram_features])

    # Decode label menggunakan label_encoder
    predicted_label = label_encoder.inverse_transform([np.argmax(predictions)])[0]

    print('Path:', file_path)
    print('Predicted label:', predicted_label)

Perbedaan durasi audio! Durasi yang diharapkan: 0.8356235827664399 detik, Durasi file: 0.6660317460317461 detik
1/1 [==============================] - 0s 107ms/step
Path: /content/drive/MyDrive/Colab Notebooks/export4.wav
Predicted label: ba
